## Parcial cálculo numérico - Cálculo Numérico
##### Antonio Cabrera Landín 3ºMAIS
---

1. Escribir una función llamada `interp` que, dado un punto $x$, una lista con los nodos de interpolación `xp`y una lista con los valores correspondientes de la función en cada nodo `fp`, evalúe el polinomio de Lagrange en el punto `x`.

Crearemos una función para mostrar el polinomio

In [43]:
def mostrar_polinomio(coeficientes, x: str):

    n = len(coeficientes)
    
    print("P(x) = ", end="")

    for i in range(n):
        if (n - i - 1) == 0:
            print(coeficientes[i], end=i < n - 1 and " + " or "\n")
        elif (n - i - 1) == 1:
            if coeficientes[i] == 1:
                print(f"{x}", end=i < n - 1 and " + " or "\n") 
            elif coeficientes[i] != 0:     
                print(f"{coeficientes[i]}{x}", end=i < n - 1 and " + " or "\n")
        else:
            if coeficientes[i] == 1:
                print(f"{x}^{n - i - 1}", end=i < n - 1 and " + " or "\n") 
            elif coeficientes[i] != 0:     
                print(f"{coeficientes[i]}{x}^{n - i - 1}", end=i < n - 1 and " + " or "\n")

Para calcular el polinomio interpolador de Lagranfe utilizaremos la forma de Newton:

$$
P(x) = f[x_0] + f[x_0, x_1](x - x_0) + \ldots + f[x_0, \ldots, x_N](x-x_0) \dots (x - x_{N-1})
$$

Los terminos $f[x_0, \ldots, x_N]$ son las diferencias divididas y cumplen:

$$
f[x_0, x_1, \ldots, x_N] = \frac{f[x_1, \ldots, x_N] - f[x_0, \ldots, x_{N-1}]}{x_N - x_0}
$$

Sabiendo esto podemos calcular las diferencias divididas del polinomio con la tabla:

$$
\begin{array}{|c|c|c|c|c|}
	\hline
	x_{0} & f[x_{0}]\\
	\hline
	x_{1} & f[x_{1}] & f[x_{0}, x_{1}]\\
	\hline
	x_{2} & f[x_{2}] & f[x_{1}, x_{2}] & f[x_{0}, x_{1}, x_{2}]\\
	\hline
    \vdots & \vdots & \vdots & \vdots & \vdots\\
    \hline
	x_{N} & f[x_{N}] & f[x_{N-1}, x_{N}] & \ldots & f[x_{0}, x_{1}, \ldots, x_{N}]\\
	\hline
\end{array}
$$

La función `tabla_lagrange` calcula las diferencias divididas del polinomio interpolador de Lagrange en forma de Newton, es decir la diagonal de la tabla:

$$
f[x_0], f[x_0, x_1], \ldots, f[x_0, \ldots, x_N]
$$

In [41]:
'''
Calcular las diferencias divididas de la forma de Newton mediante la tabla del polinomio de Lagrange

Parámetros:
    - fx: vector de valores de la función en los nodos de interpolación
    - x: vector de nodos de interpolación

Retorno:
    - diferencias_divididas: vector con las diferencias divididas de la forma de Newton
'''
def tabla_lagrange(x, fx):

    # Inicializar la tabla
    tabla = np.zeros((len(x), len(x)))
    
    # Damos los valores iniciales de la tabla
    for i in range(len(x)):
        tabla[i, 0] = fx[i]

    # Calcular la tabla
    for j in range(1, len(x)):
        for i in range(j, len(x)):
            tabla[i, j] = (tabla[i, j - 1] - tabla[i - 1, j - 1]) / (x[i] - x[i - j])

    # Extraer la diagonal principal de la tabla
    diferencias_divididas = np.diag(tabla)

    mostrar_polinomio(diferencias_divididas, x="x")

    return diferencias_divididas

Con esto ya podemos implementar la función `interp` que evaluará el polinomio interpolador en un punto `x`, para ello tenemos que implementar la fórmula:

$$
P(x) = f[x_0] + f[x_0, x_1](x - x_0) + \ldots + f[x_0, \ldots, x_N](x-x_0) \dots (x - x_{N-1})
$$

Para ahorrar costo computacional usaremos una variable auxiliar `aux` en el que guardaremos el valor anterior a la multiplicación $(x-x_0) \dots (x - x_{N-1})$, ya que si no tendríamos que calcular los terminos anteriores en cada iteración del bucle.

In [36]:
'''
Evaluar el polinomio de Lagrange en el punto x mediante las diferencias divididas de la forma de Newton

Parámetros:
    - x: punto en el que se quiere evaluar el polinomio interpolador
    - xp: vector de nodos de interpolación
    - fx: vector de valores de la función en los nodos de interpolación

Retorno:
    - px: valor del polinomio interpolador en el punto x
'''
def interp(x, xp, fx):
    diferencias_divididas = tabla_lagrange(xp, fx) 
    px = 0
    aux = 1
    xp = xp.sort()
    
    for j in range(len(xp)):
        px += diferencias_divididas[j] * aux
        aux *= (x - xp[j])

    return px